In [ ]:
import os
import sys
import time
import numpy as np
import pandas as pd
import datetime as dtm
import matplotlib.pyplot as plt

# Import FMLC
#sys.path.append(r'C:\Users\Christoph\Documents\PublicRepos\FMLC')
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath("Test.ipynb"))))
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack

![Architecture](../documentation/images/architecture.jpg)

In [ ]:
from Controllers_Definition import communication_dummy, forecaster_dummy, controller_dummy

In [ ]:
# Setup control modules
controller = {}
# Slow loop (5 s sampletime)
controller['getweather'] = {'fun':communication_dummy, 'sampletime':5}
controller['getgrid'] = {'fun':communication_dummy, 'sampletime':'getweather'}
controller['forecast'] = {'fun':forecaster_dummy, 'sampletime':'getgrid'}
controller['setgrid'] = {'fun':communication_dummy, 'sampletime':'forecast'}
# Fast loop (1 s sampletime)
controller['getscada'] = {'fun':communication_dummy, 'sampletime':1}
controller['controller'] = {'fun':controller_dummy, 'sampletime':'getscada'}
controller['setscada'] = {'fun':communication_dummy, 'sampletime':'controller'}

# Link control
mapping = {}
# Slow loop
mapping['getweather_mode'] = 'get_weather'
mapping['getgrid_mode'] = 'get_iso'
mapping['forecast_wf'] = 'getweather_data'
mapping['forecast_scada'] = 'getscada_data'
mapping['setgrid_mode'] = 'set_iso'
# Fast loop
mapping['getscada_mode'] = 'get_scada'
mapping['controller_data'] = 'forecast_data'
mapping['setscada_mode'] = 'set_scada'

### Run MGC

In [ ]:
mgc = controller_stack(controller, mapping, tz=-8, debug=True, parallel=False)
for i in range(10):
    mgc.query_control(time.time())
    time.sleep(0.2)
log = mgc.log_to_df()
print('Log-keys:', list(log.keys()))
print('Log-messages from "controller"')
print(log['controller'])
df = pd.read_json(log['controller']['data'].iloc[-1])
df['P_batt'] = pd.DataFrame(log['controller']['control'][-1])
df.plot()
plt.show()